In [ ]:
from __future__ import division
import os
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
import cv2
import visdom
from models.darknet import Darknet, write_results
from data import prep_image, load_classes
from torchvision.transforms import ToTensor, Lambda
viz = visdom.Visdom()
dnet = Darknet("cfg/yolov3.cfg")
weight_file = 'checkpoints/yolov3.weights'
dnet.load_net_binary(weight_file)
dnet.eval()

test_dir = 'data/test_imgs'
class_names = load_classes("data/coco.names")
image_names = [os.path.join(test_dir, n_) for n_ in os.listdir(test_dir)]

SyntaxError: invalid syntax (imio.py, line 22)

In [ ]:
cv_images = list(map(cv2.imread, image_names))
input_images = [prep_image(im_, 416) for im_ in cv_images]
cv_img_to_tensor_prep = ToTensor()
raw_ims = [prep(im_)[[2, 1, 0]] for im_ in cv_images]

i = 0
pred = dnet(input_images[i])

corners, clss, scores = write_results(pred, 0.5)
oim = mark_box(cv_images[0].copy(), corners[0], clss[0], scores[0], class_names)
oim_ = prep(oim)[[2, 1, 0]]
imwnd0 = viz.image(oim_, opts=dict(title='Random!', caption='How random.'),)


In [20]:
class CoordConvertor:
    def __init__(self, x0, y0, r):
        self.x0 = x0
        self.y0 = y0
        self.r  = r
        
    def origin_to_letterbox(self, x, y):
        return x*self.r + self.x0, y*self.r + self.y0
    
    def letterbox_to_origin(self, x, y):
        return (x-self.x0)/self.r, (y-self.y0)/self.r
    
def get_coordinate_converter(in_dim, target_dim):
    """
    If you use letter-box image padding and resizing, how to convert
    coordinates back and forth between two images.
    """
    in_w, in_h = in_dim
    if len(target_dim) == 2:
        out_w, out_h = target_dim
    else:
        out_w = out_h = target_dim
    resize_ratio_w = float(out_w) / in_w
    resize_ratio_h = float(out_h) / in_h
    if resize_ratio_w < resize_ratio_h:
        resize_ratio = resize_ratio_w
        outim_w = out_w
        outim_h = in_h * resize_ratio_w # < target_dim
    else:
        resize_ratio = resize_ratio_h
        outim_h = out_h
        outim_w = in_w * resize_ratio_h # < target_dim
        
    # original_to_letterbox
    h0 = (out_h - outim_h) // 2
    w0 = (out_w - outim_w) // 2
    return CoordConvertor(w0, h0, resize_ratio)

def to_letterbox_image(img, target_dim):
    """
    Resize image with unchanged aspect ratio using padding.

    :param img: HxWxC
    :param target_dim: target_w and target_h
    """    
    in_w, in_h = img.shape[1], img.shape[0]
    if len(target_dim) == 2:
        out_w, out_h = target_dim
    else:
        out_w = out_h = target_dim
    f = get_coordinate_converter((in_w, in_h), target_dim)
    outim_h = round(in_h*f.r)
    outim_w = round(in_w*f.r)
    resized_image = cv2.resize(img, (round(in_w*f.r), round(in_h*f.r)),
                               interpolation=cv2.INTER_CUBIC)

    canvas = np.full((out_h, out_w, 3), 128)
    canvas[f.y0:f.y0+outim_h, f.x0:f.x0+outim_w, :] = resized_image

    return canvas

In [21]:
im = cv2.imread('data/test_imgs/dog.jpg')
im_ = to_letterbox_image(im, (416, 416))

TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
%debug

> <ipython-input-20-9ca9bec55b1c>(58)to_letterbox_image()
     56 
     57     canvas = np.full((out_h, out_w, 3), 128)
---> 58     canvas[f.y0:f.y0+outim_h, f.x0:f.x0+outim_w, :] = resized_image
     59 
     60     return canvas

ipdb> f
ipdb> f.y0
52.0
ipdb> 
52.0
ipdb> f.x0
0


In [ ]:
def to_letterbox_image_backup(img, target_dim):
    """
    Resize image with unchanged aspect ratio using padding.

    :param img: HxWxC
    :param target_dim: target_w and target_h
    """    
    in_w, in_h = img.shape[1], img.shape[0]
    f, _, r = get_coordinate_converters((in_w, in_h), target_dim)
    
    if len(target_dim) == 2:
        out_w, out_h = target_dim
    else:
        out_w = out_h = target_dim
    resize_ratio_w = float(out_w) / in_w
    resize_ratio_h = float(out_h) / in_h
    if resize_ratio_w < resize_ratio_h:
        resize_ratio = resize_ratio_w
        outim_w = out_w
        outim_h = in_h * resize_ratio_w # < target_dim
    else:
        resize_ratio = resize_ratio_h
        outim_h = out_h
        outim_w = in_w * resize_ratio_h # < target_dim
        
    resized_image = cv2.resize(img, (outim_w, outim_h),
                               interpolation=cv2.INTER_CUBIC)

    canvas = np.full((out_w, out_h, 3), 128)

    h0 = (out_h - outim_h) // 2
    w0 = (out_w - outim_w) // 2
    canvas[h0:h0+outim_h, w0:w0+outim_w, :] = resized_image

    return canvas

def get_coordinate_converters(in_dim, target_dim):
    """
    If you use letter-box image padding and resizing, how to convert
    coordinates back and forth between two images.
    """
    in_w, in_h = in_dim
    if len(target_dim) == 2:
        out_w, out_h = target_dim
    else:
        out_w = out_h = target_dim
    resize_ratio_w = float(out_w) / in_w
    resize_ratio_h = float(out_h) / in_h
    if resize_ratio_w < resize_ratio_h:
        resize_ratio = resize_ratio_w
        outim_w = out_w
        outim_h = in_h * resize_ratio_w # < target_dim
    else:
        resize_ratio = resize_ratio_h
        outim_h = out_h
        outim_w = in_w * resize_ratio_h # < target_dim
        
    # original_to_letterbox
    h0 = (out_h - outim_h) // 2
    w0 = (out_w - outim_w) // 2
    o2l = lambda x,y: x*resize_ratio + x0, y*resize_ratio + y0
    l2o = lambda x,y: (x-x0)/resize_ratio, (y-y0)/resize_ratio
    return o2l, l2o, resize_ratio

    